In [ ]:
import numpy as np
import pandas as pd 
from pylab import mpl, plt
import random
import optuna

import torch
import torch.nn as nn
import torch.optim as optim


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

import warnings
warnings.simplefilter("ignore", UserWarning)

import vectorbtpro as vbt



In [ ]:


# Seed for Python's random module
random.seed(42)

# Seed for NumPy's random number generator
np.random.seed(42)

# Seed for PyTorch
torch.manual_seed(42)

# Additional settings for multi-threaded reproducibility
torch.use_deterministic_algorithms(True)  # This is a newer way compared to 'torch.backends.cudnn.deterministic'


In [ ]:
df = pd.read_csv('2ySOLdata1h.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.set_index('timestamp', inplace=True)
pd.set_option('future.no_silent_downcasting', True)
df_filtered = df[df['signal'] != 'SignalNone']

# Iterate through the DataFrame and adjust the signals
for i in range(1, len(df_filtered)):
    current_signal = df_filtered.iloc[i]['signal']
    previous_signal = df_filtered.iloc[i - 1]['signal']
    current_close = df_filtered.iloc[i]['Close']
    previous_close = df_filtered.iloc[i - 1]['Close']
    
    if current_signal == previous_signal:
        if current_signal == 'SignalLong' and previous_close > current_close:
            df_filtered.iloc[i - 1, df_filtered.columns.get_loc('signal')] = 'SignalNone'
        elif current_signal != 'SignalLong' and previous_close < current_close:
            df_filtered.iloc[i - 1, df_filtered.columns.get_loc('signal')] = 'SignalNone'
        else:
            df_filtered.iloc[i, df_filtered.columns.get_loc('signal')] = 'SignalNone'


df.update(df_filtered)

# # Assuming df is your DataFrame
# previous_signal = None  # Initialize a variable to keep track of the previous non-"SignalNone" value

# for i in range(len(df)):
#     if df.iloc[i, df_filtered.columns.get_loc('signal')] == "SignalNone" and previous_signal is not None:
#         df.iloc[i, df_filtered.columns.get_loc('signal')] = previous_signal  # Replace "SignalNone" with the previous signal
#     elif df.iloc[i, df_filtered.columns.get_loc('signal')] != "SignalNone":
#         previous_signal = df.iloc[i, df_filtered.columns.get_loc('signal')]  # Update the previous signal to the current one if it's not "SignalNone"

# df = df.loc[df['signal'] != 'SignalNone']

df['signal'] = df['signal'].replace({'SignalLong': 2, 'SignalShort': 0, 'SignalNone': 1})
df = df.ffill()

In [ ]:
data = vbt.Data.from_data(df)
features = data.run("talib", mavp=vbt.run_arg_dict(periods=14))
data.data['symbol'] = pd.concat([data.data['symbol'], features], axis=1)
data.data['symbol'].drop(['Open', 'High', 'Low'], axis=1, inplace=True)
# This will drop columns from the DataFrame where all values are NaN
data.data['symbol'] = data.data['symbol'].dropna(axis=1, how='all')

open_price = data.get('Open')
high_price = data.get('High')
low_price = data.get('Low')
close_price = data.get('Close')

data.data['symbol'] = data.data['symbol'].dropna()
predictor_list = data.data['symbol'].drop('signal', axis=1).columns.tolist()


X = data.data['symbol'][predictor_list]

y = data.data['symbol']['signal']

X.columns = X.columns.astype(str)

In [ ]:
# Split the data into a training set and a test set
# Assuming X is a DataFrame or a NumPy array
indices = np.arange(X.shape[0])

# First, split your data into a training+validation set and a separate test set
X_train_val, X_test, y_train_val, y_test, indices_train_val, indices_test = train_test_split(X, y, indices, test_size=0.3, shuffle=False)

# Then, split the training+validation set into a training set and a validation set
X_train, X_val, y_train, y_val, indices_train, indices_val = train_test_split(X_train_val, y_train_val, indices_train_val, test_size=0.2, shuffle=False)  # 0.2 here means 20% of the original data, or 25% of the training+validation set

# Now, `indices_val` holds the indices of your original dataset that were used for the validation set.


scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.fit_transform(X_val)

In [ ]:
timestep = 20

def create_sequences(input_data, timestep):
    sequences = []
    data_len = len(input_data)
    for i in range(data_len - timestep):
        seq = input_data[i:(i + timestep)]
        sequences.append(seq)
    return np.array(sequences)

X_train_list = create_sequences(X_train_scaled, timestep)
X_val_list = create_sequences(X_val_scaled, timestep)
X_test_list = create_sequences(X_test_scaled, timestep)
y_train_seq_ar = y_train[timestep:]
y_val_seq_ar = y_val[timestep:]
y_test_seq_ar = y_test[timestep:]

In [ ]:
# Convert to numpy arrays
x_train_ar = np.array(X_train_list)
y_train_seq = np.array(y_train_seq_ar).astype(int)
x_val_ar = np.array(X_val_list)  
y_val_seq = np.array(y_val_seq_ar).astype(int)
x_test_ar = np.array(X_test_list)  
y_test_seq = np.array(y_test_seq_ar).astype(int)

In [ ]:
# Convert to tensors
X_train_tensor = torch.tensor(x_train_ar, dtype=torch.float32) # .to(device)
y_train_tensor = torch.tensor(y_train_seq, dtype=torch.long)
X_val_tensor = torch.tensor(x_val_ar, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val_seq, dtype=torch.long)
X_test_tensor = torch.tensor(x_test_ar, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_seq, dtype=torch.long)

In [ ]:
# Check for MPS (GPU on M1 Mac) availability and set it as the device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [ ]:
# Compute class weights
# Convert y_train to a numpy array if it's a tensor
if isinstance(y_train_seq, torch.Tensor):
    y_train_seq_np = y_train_seq.cpu().numpy()
else:
    y_train_seq_np = y_train_seq  # Assuming y_train_seq is already a numpy array or similar

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_seq_np), y=y_train_seq_np)
# Decrease the weight of the '1' class
decrease_factor = 0.1  # Adjust this factor as needed
class_weights[1] *= decrease_factor

class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)


# Move class weights to the same device as your model and data
class_weights_tensor = class_weights_tensor.to(device)  # device could be 'cpu' or 'cuda'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.hidden_dim = hidden_dim
        self.linear = nn.Linear(hidden_dim, 1)

    def forward(self, lstm_output):
        # lstm_output shape: [batch_size, seq_length, hidden_dim]
        weights = torch.tanh(self.linear(lstm_output))
        weights = F.softmax(weights, dim=1)
        
        # Context vector with weighted sum
        context = weights * lstm_output
        context = torch.sum(context, dim=1)
        return context, weights

class BiLSTMClassifierWithAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout_rate):
        super(BiLSTMClassifierWithAttention, self).__init__()
        
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        
        # Convolutional Layer
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=hidden_dim, kernel_size=3, stride=1, padding=1)
        
        # Batch Normalization Layer for Conv1d
        self.bn_conv1 = nn.BatchNorm1d(hidden_dim)
        
        # LSTM Layer
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        
        # Attention Layer
        self.attention = Attention(hidden_dim * 2)  # For bidirectional LSTM
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)
        
        # Fully connected layers
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)  # Adjusted for attention context vector
        
        # Batch Normalization Layer for FC1
        self.bn_fc1 = nn.BatchNorm1d(hidden_dim)
        
        self.fc2 = nn.Linear(hidden_dim, output_dim)  # Output layer
        
        # Additional Dropout for the fully connected layer
        self.dropout_fc = nn.Dropout(dropout_rate / 2)

    def forward(self, x):
        # Reshape x for Conv1d
        x = x.permute(0, 2, 1)
        
        # Convolutional layer
        x = self.conv1(x)
        x = self.bn_conv1(x)
        x = F.relu(x)
        
        # Reshape back for LSTM
        x = x.permute(0, 2, 1)
        
        # Initialize hidden and cell states
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(x.device)
        
        # LSTM layer
        out, _ = self.lstm(x, (h0, c0))
        
        # Applying attention mechanism to LSTM outputs
        context, _ = self.attention(out)
        
        # Fully connected layers using the context vector from attention
        out = self.fc1(context)
        out = self.bn_fc1(out)
        out = F.relu(out)
        out = self.dropout_fc(out)
        out = self.fc2(out)
        
        return out


In [ ]:
def validate_multi_with_metrics(model, criterion, X_val, y_val, device):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        output = model(X_val)
        predicted_probs = torch.softmax(output, dim=1)
        predictions = torch.argmax(predicted_probs, dim=1)  # Get the index of the max log-probability as the prediction
        
        loss = criterion(output, y_val)  # Ensure y_val is of dtype long and contains class indices

        # Convert to CPU and numpy for sklearn metrics
        predictions_np = predictions.cpu().numpy()
        y_val_np = y_val.cpu().numpy()

        accuracy = accuracy_score(y_val_np, predictions_np)
        precision = precision_score(y_val_np, predictions_np, average='weighted')
        recall = recall_score(y_val_np, predictions_np, average='weighted')
        f1 = f1_score(y_val_np, predictions_np, average='weighted')

    model.train()  # Set back to train mode
    return {
        'loss': round(loss.item(), 4),
        'accuracy': round(accuracy, 4),
        'precision': round(precision, 4),
        'recall': round(recall, 4),
        'f1': round(f1, 4)
    }

In [ ]:
def validate_financials(model, X_val_selected_gpu):
    model.eval()
    with torch.no_grad():
        y_test_pred = model(X_val_selected_gpu)
        probabilities = torch.sigmoid(y_test_pred).squeeze()  # Apply sigmoid to convert logits to probabilities
        predicted_labels = (probabilities > 0.5).long()  # Threshold probabilities to get binary predictions
        predicted_labels_numpy = predicted_labels.cpu().numpy()

    # Use predicted labels to simulate a trading strategy
    adjusted_indices_val = indices_val[timestep:]
    adjusted_indices_test = indices_test[timestep:] 
    
    df_split = data.data['symbol'].iloc[adjusted_indices_val].copy()
    df_split.loc[:, "signal"] = predicted_labels_numpy
    signal = df_split['signal']
    entries = signal == 1
    exits = signal == 0
    pf = vbt.Portfolio.from_signals(
        close=df_split.Close, 
        long_entries=entries, 
        short_entries=exits,
        size=100,
        size_type='value',
        init_cash='auto'
    )
    stats = pf.stats()
    total_return = stats['Total Return [%]']
    orders = stats['Total Orders']
    calmer_ratio = stats['Calmar Ratio']
    
    model.train()
    return {
        "orders": orders,
        "calmer_returns": (calmer_ratio+total_return)
    }

In [ ]:
import plotly.graph_objects as go
from collections import deque

vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 500
vbt.settings['plotting']['layout']['height'] = 250

def objective(trial, type, hparams):


    if hparams == 'static':
        # Suggest hyperparameters
        hidden_dim = 32 # trial.suggest_categorical('hidden_dim', [16, 32, 64])
        num_layers = 2 # trial.suggest_int('num_layers', 1, 3)
        lr = 1e-2 # trial.suggest_float('lr', 1e-5, 1e-1, log=True)
        step_size = 25 # trial.suggest_int('step_size', 10, 100)
        gamma = 0.85 # trial.suggest_float('gamma', 0.85, 0.99)
        dropout_rate = 0.1 # trial.suggest_float('dropout_rate', 0.1, 0.4)
    elif hparams == 'dynamic':
        # Suggest hyperparameters
        hidden_dim = trial.suggest_categorical('hidden_dim', [16, 32, 64])
        num_layers = trial.suggest_int('num_layers', 1, 3)
        lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
        step_size = trial.suggest_int('step_size', 10, 100)
        gamma = trial.suggest_float('gamma', 0.85, 0.99)
        dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.4)

    if type == 'single':
        # Use only the selected feature to create new tensors
        feature_idx = 89 # trial.suggest_int('feature_idx', 0, X_train_tensor.shape[2] - 1)
        X_train_selected = X_train_tensor[:, :, 0:X_train_tensor.shape[2] - 1]
        X_val_selected = X_val_tensor[:, :, 0:X_train_tensor.shape[2] - 1]
        # X_test_selected = X_test_tensor[:, :, feature_idx:feature_idx+1]
    elif type == 'tophalf':
        # Use only the selected feature to create new tensors
        feature_idx = trial.suggest_categorical('feature_idx', unique_features_0)
        X_train_selected = X_train_tensor[:, :, feature_idx:feature_idx+1]
        X_val_selected = X_val_tensor[:, :, feature_idx:feature_idx+1]
    elif type == 'multi':
        # Suggest a boolean flag for each feature to decide if it should be included
        included_features = [trial.suggest_categorical(f'include_feature_{i}', [True, False]) for i in unique_features]
        included_features_idx = [i for i, f in enumerate(included_features) if f]
        # If no features are selected, we can either skip this trial or select a default feature
        if not included_features_idx:
            return None  # Or handle this case as you see fit
        # Use a selection of features to create new tensors
        X_train_selected = X_train_tensor[:, :, included_features_idx]
        X_val_selected = X_val_tensor[:, :, included_features_idx]
        X_test_selected = X_test_tensor[:, :, included_features_idx]
    elif type == 'all':
        # Use only the selected features to create new tensors
        X_train_selected = X_train_tensor[:, :, top_group_feature_indices]
        X_val_selected = X_val_tensor[:, :, top_group_feature_indices]
        X_test_selected = X_test_tensor[:, :, top_group_feature_indices] 
        

    
    
    X_train_selected_gpu = X_train_selected.float().to(device)
    X_val_selected_gpu = X_val_selected.float().to(device)
    y_train_tensor_gpu = y_train_tensor.float().to(device)
    y_val_tensor_gpu = y_val_tensor.float().to(device)

    # Initialize model and move it to the MPS device
    model = BiLSTMClassifierWithAttention(input_dim=X_train_selected.shape[-1], hidden_dim=hidden_dim, num_layers=num_layers, output_dim=len(np.unique(y_train_tensor.cpu().numpy())), dropout_rate=dropout_rate).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)



    rolling_window = deque(maxlen=rolling_window_size)
    epoch_nums_1 = []
    calmer_return = []
    val_loss = []
    accuracy = []
    precision = []
    recall = []
    f1 = []
    # auc = []
    
    
    
    best_calmer_return = float('-inf')  # Initialize with the lowest possible number

    # Training loop
    model.train()
    for epoch in range(num_epochs):  # use a small number of epochs for demonstration
        optimizer.zero_grad()
        output = model(X_train_selected_gpu)
        output = torch.squeeze(output)  # This removes the extra dimension
        loss = criterion(output, y_train_tensor_gpu)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        if epoch % validation_frequency == 0:
            # financial_results = validate_financials(model, X_val_selected_gpu)
            # current_calmer_return = financial_results['calmer_returns']
            validation_results = validate_multi_with_metrics(model, criterion, X_val_selected_gpu, y_val_tensor_gpu, device)

        # # Append metrics for plotting
            epoch_nums_1.append(epoch)
            val_loss.append(validation_results['loss'])
            accuracy.append(validation_results['accuracy'])
            precision.append(validation_results['precision'])
            recall.append(validation_results['recall'])
            f1.append(validation_results['f1'])
            # auc.append(validation_results['auc'])
            # calmer_return.append(current_calmer_return)
        
            # Append the new return to the rolling window
            # rolling_window.append(current_calmer_return)

            # if len(rolling_window) == rolling_window_size:
            #     variance = np.var(list(rolling_window))
                
            #     if hparams == 'dynamic':
            #         # Save the model if the current return is a new maximum
            #         if current_calmer_return > best_calmer_return:
            #             best_calmer_return = current_calmer_return
            #             # Save the model
            #             torch.save(model.state_dict(), f'model_epoch_{epoch}_calmer_{current_calmer_return:.4f}.pth')
            #             print(f"New best model saved at epoch {epoch} with Calmer Return: {current_calmer_return}")
                    
            #     # Check if the variance is below the threshold
            #     if variance < variance_threshold:
            #         print(f"Early stopping triggered at epoch {epoch}. Variance of returns over the last {rolling_window_size} validation steps is below the threshold.")
            #         break
                
            #     # Check if the last rolling_window_size returns are negative
            #     if len(calmer_return) >= rolling_window_size and all(x < 0 for x in calmer_return[-rolling_window_size:]):
            #         print(f"Early stopping triggered at epoch {epoch}. The last {rolling_window_size} returns are negative.")
            #         break
        
    model.eval()
    with torch.no_grad():
        y_test_pred = model(X_val_selected_gpu)
        probabilities = torch.softmax(y_test_pred, dim=1)
        _, predicted_labels = torch.max(probabilities, 1)
        predicted_labels_numpy = predicted_labels.cpu().numpy()

    # Use predicted labels to simulate a trading strategy
    adjusted_indices_val = indices_val[timestep:]
    adjusted_indices_test = indices_test[timestep:] 
    df_split = data.data['symbol'].iloc[adjusted_indices_val].copy()


    df_split.loc[:, "signal"] = predicted_labels_numpy
    signal = df_split['signal']
    entries = signal == 2
    exits = signal == 0
    pf = vbt.Portfolio.from_signals(
        close=df_split.Close, 
        long_entries=entries, 
        short_entries=exits,
        size=100,
        size_type='value',
        init_cash='auto'
    )
    
    stats = pf.stats()
    total_return = round(stats['Total Return [%]'], 2)
    orders = stats['Total Orders']
    calmer_ratio = stats['Calmar Ratio']
    

    pf_bt_fig = pf.plot({"orders", "cum_returns"})
    validation_metrics_fig = go.Figure()

    validation_metrics_fig.add_trace(go.Scatter(x=epoch_nums_1, y=val_loss, mode='lines+markers', name='val_loss'))
    validation_metrics_fig.add_trace(go.Scatter(x=epoch_nums_1, y=accuracy, mode='lines+markers', name='accuracy'))
    validation_metrics_fig.add_trace(go.Scatter(x=epoch_nums_1, y=precision, mode='lines+markers', name='precision'))
    validation_metrics_fig.add_trace(go.Scatter(x=epoch_nums_1, y=recall, mode='lines+markers', name='recall'))
    validation_metrics_fig.add_trace(go.Scatter(x=epoch_nums_1, y=f1, mode='lines+markers', name='f1'))
    # validation_metrics_fig.add_trace(go.Scatter(x=epoch_nums_1, y=auc, mode='lines+markers', name='auc'))

    validation_metrics_fig.update_layout(
        template='plotly_dark',
        autosize=False,
        width=700,  # Set the width of the figure
        height=150,  # Set the height of the figure
        margin=dict(l=10, r=10, b=10, t=30, pad=5),
        title='Calmer Ratio + Returns Over Epochs'  # You can set the title directly here
    )


    # if orders < 6:
    #     print(f"Only {orders} trades were made")
    #     calmer_ratio = 0.0
    #     total_return = 0.0
    # else:
    #     if (calmer_ratio + total_return) > 0:
    pf_bt_fig.show()    
    validation_metrics_fig.show()
    print(f"Total returns: {total_return} %")
        
        
    # # Return the total return as the objective to maximize it
    return round((calmer_ratio + total_return), 2)



In [ ]:
# Before running the study, ensure your data tensors are on the CPU as Optuna will handle moving them to the GPU

X_train_tensor = X_train_tensor.cpu()
y_train_tensor = y_train_tensor.cpu()
X_val_tensor = X_val_tensor.cpu()
y_val_tensor = y_val_tensor.cpu()
X_test_tensor = X_test_tensor.cpu()
y_test_tensor = y_test_tensor.cpu()



In [ ]:
num_epochs = 100
validation_frequency = 10 # how often (epochs) to run validation

rolling_window_size = 15 # how many validation freqs to consider for variance
variance_threshold = 3 # below this threshold, we stop training

num_trials_0 = 5 # X_train_tensor.shape[2] * 1.2  # number of trials to run

study_0 = optuna.create_study(direction='maximize')
study_0.optimize(lambda trial: objective(trial, "single", "static"), n_trials=num_trials_0)

print('Best trial:', study_0.best_trial.params)


In [ ]:
completed_trials_0 = [trial for trial in study_0.trials if trial.state == optuna.trial.TrialState.COMPLETE]

# Sort the trials based on their performance (assuming higher return is better)

sorted_trials_0 = sorted(completed_trials_0, key=lambda trial: trial.value, reverse=True)

# Get the top N performing feature indices
top_n_0 = int(len(completed_trials_0)/2)  # For example, top 5 features
top_n_features_0 = [trial.params['feature_idx'] for trial in sorted_trials_0[:top_n_0]]

# # print("Top performing feature indices:", top_n_features)

# Map the indices to names
top_n_feature_names_0 = [predictor_list[idx] for idx in top_n_features_0]
print(f"Top performing features: {top_n_feature_names_0}")
print(f"Top performing indicies: {top_n_features_0}")

# Remove duplicates without preserving order
unique_features_0 = list(set(top_n_features_0))

print(unique_features_0)

In [ ]:
num_trials_1 = len(unique_features_0) * 1.3  # number of trials to run

study_1 = optuna.create_study(direction='maximize')
study_1.optimize(lambda trial: objective(trial, "tophalf", "static"), n_trials=num_trials_1)

print('Best trial:', study_1.best_trial.params)

In [ ]:
completed_trials = [trial for trial in study_1.trials if trial.state == optuna.trial.TrialState.COMPLETE]


sorted_trials = sorted(completed_trials, key=lambda trial: trial.value, reverse=True)

# Get the top N performing feature indices
top_n = 20  # For example, top 5 features
top_n_features = [trial.params['feature_idx'] for trial in sorted_trials[:top_n]]

# # print("Top performing feature indices:", top_n_features)

# Map the indices to names
top_n_feature_names = [predictor_list[idx] for idx in top_n_features]
print(f"Top performing features: {top_n_feature_names}")
print(f"Top performing indicies: {top_n_features}")

# Remove duplicates without preserving order
unique_features = list(set(top_n_features))

print(unique_features)


In [ ]:
num_trials_2 = 100  # number of trials to run

study_2 = optuna.create_study(direction='maximize')
study_2.optimize(lambda trial: objective(trial, "multi", "static"), n_trials=num_trials_2)

print('Best trial:', study_2.best_trial.params)

In [ ]:
best_trial_2 = study_2.best_trial

print(f"Best trial number: {best_trial_2.number}")
print("Best trial's parameters:", best_trial_2.params)
print("Best trial's objective value:", best_trial_2.value)

# Assuming best_trial.params is your dictionary
params_2 = best_trial_2.params

# Extracting feature indices for which the value is True
top_group_feature_indices = [int(key.split('_')[-1]) for key, value in params_2.items() if value]

print("Included feature indices:", top_group_feature_indices)

# Map the indices to names
top_group_feature_names = [predictor_list[idx] for idx in top_group_feature_indices]

print("Top performing feature names:", top_group_feature_names)

In [ ]:
num_trials_3 = 2

study_3 = optuna.create_study(direction='maximize')
study_3.optimize(lambda trial: objective(trial, "all", "dynamic"), n_trials=num_trials_3)

print(f'Best trial:', study_3.best_trial.number, study_3.best_trial.params)

In [ ]:
best_trial_3 = study_3.best_trial

print(f"Best trial number: {best_trial_3.number}")
print("Best trial's parameters:", best_trial_3.params)
print("Best trial's objective value:", best_trial_3.value)

# Assuming best_trial.params is your dictionary
params_3 = best_trial_3.params
params_3


In [ ]:
X_train_tensor = X_train_tensor.cpu()
y_train_tensor = y_train_tensor.cpu()
X_val_tensor = X_val_tensor.cpu()
y_val_tensor = y_val_tensor.cpu()
X_test_tensor = X_test_tensor.cpu()
y_test_tensor = y_test_tensor.cpu()